In [1]:
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

# Contrastive_learning_for_unseen_environments

In [3]:
# random seed
np.random.seed(1024)
torch.manual_seed(1024)

# gpu setting
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device = DEVICE
torch.cuda.set_device(DEVICE)

## 0. Setup 

### a. data

In [4]:
from data.spectrogram import import_data, import_pair_data
from data import *

### b. model setup

In [5]:
from models.baseline import *
from models.cnn import *
from models.self_supervised import *
from models.utils import *
from models import *

In [6]:
from losses import NT_Xent
from contrastive_learning import pretrain
from train import record_log,save,evaluation,train,load_checkpoint

----------------------EXP  19 ----------------------


In [7]:
def initial_filtering_activities(datas, activities):
    for activity in activities:
        datas = where(*datas,condition=(datas[-1] != activity))
    return datas

In [8]:
def create_vgg16(output_size=(2,2)):
    """
    VGG 16 for 1 channel image, output: 512*output_size
    """
    net = vgg16()
    net = torch.nn.Sequential(Stack(),
                                *(list(net.children())[:-2]),
                                nn.AdaptiveAvgPool2d(output_size),
                                Flatten())
    out_size = 512*output_size[0]*output_size[1]
    return net, out_size

In [9]:
def create_encoder(network,pairing):
    if network == "shallow":
        if pairing == 'csi':
            encoder = create_baseline_encoder(scale_factor=1)
            outsize = 960
        elif pairing == 'nuc2':
            encoder = create_baseline_encoder(scale_factor=1)
            outsize = 960
        elif pairing == 'pwr':
            encoder = create_baseline_encoder(scale_factor=3)
            outsize = 1152
        else: 
            raise ValueError("pairing must be in {'csi','nuc2','pwr'}")
    elif network == "alexnet":
        if pairing == 'csi':
            encoder,outsize = create_alexnet((1,4),scale_factor=1)
        elif pairing == 'nuc2':
            encoder,outsize = create_alexnet((1,4),scale_factor=1)
        elif pairing == 'pwr':
            encoder,outsize = create_alexnet((4,1),scale_factor=2)
        else: 
            raise ValueError("pairing must be in {'csi','nuc2','pwr'}")
    elif network == "resnet":
        if pairing == 'csi':
            encoder,outsize = create_resnet18((2,2))
        if pairing == 'nuc2':
            encoder,outsize = create_resnet18((2,2))
        elif pairing == 'pwr':
            encoder,outsize = create_resnet18((2,2))
        else: 
            raise ValueError("pairing must be in {'csi','nuc2','pwr'}")
    elif network == "vgg16":
        if pairing == 'csi':
            encoder,outsize = create_vgg16((2,2))
        if pairing == 'nuc2':
            encoder,outsize = create_vgg16((2,2))
        elif pairing == 'pwr':
            encoder,outsize = create_vgg16((2,2))
        else: 
            raise ValueError("pairing must be in {'csi','nuc2','pwr'}")
    else:
        raise ValueError("network must be in {'shallow','alexnet','resnet'}")
    return encoder, outsize

In [10]:
def record_log(record_outpath,exp_name,phase,record='None',cmtx='None',cls='None',loss_rec=True,acc_rec=False):
    prefix = record_outpath+'/'+exp_name+'_Phase_'+phase
    if type(record) != str:
        if loss_rec:
            pd.DataFrame(record['loss'],columns=['loss']).to_csv(prefix+'_loss.csv')
        if acc_rec: 
            df = pd.concat((pd.DataFrame(record['accuracy']),
                            pd.DataFrame(record['f1_weighted']),
                            pd.DataFrame(record['f1_macro'])),
                            axis=1)
            df.columns = ['accuracy','f1_weighted','f1_macro']
            df.to_csv(prefix+'_accuracy.csv')
    if type(cmtx) != str:
        cmtx.to_csv(prefix+'_cmtx.csv')
    if type(cls) != str:
        cls.to_csv(prefix+'_report.csv')
    return

In [11]:
def save_model(model_outpath,exp_name,phase,model):
    model_fp = model_outpath+'/'+exp_name+'_Phase_'+phase
    torch.save(encoder.state_dict(), model_fp)
    return model_fp

def load_encoder(network, model_fp):
    encoder, outsize = create_encoder(network,'nuc2')
    encoder.load_state_dict(torch.load(model_fp))
    return encoder, outsize

def load_model(network, freeze, lb, model_fp):
    encoder, outsize = create_encoder(network,'nuc2')
    model = add_classifier(encoder,in_size=outsize,out_size=len(lb.classes_),freeze=freeze)
    model.load_state_dict(torch.load(model_fp))
    return model
    


# Main

In [28]:
TRAINMODE = 'simclr' # 'normal' #        
NETWORK =  'vgg16' # 'resnet' # 'alexnet' # 'shallow' #      
PAIRING =  'nuc2' # 'pwr' #  
TEMPERATURE = 'L' 

REGULARIZE = None
JOINT = 'first'

PRETRAIN_SKIP = [] # 'standff'
FINETUNE_SKIP = [] # 'waving','waving'

EXTRA = None

In [29]:
fp2 = './data/experiment_data/exp_2/spectrogram' 
fp3 = './data/experiment_data/exp_3/spectrogram' 
fp4 = './data/experiment_data/exp_4/spectrogram_multi'
OUT_PATH = './laboratory/experiment_b'

BATCH_SIZE = 64
NUM_WORKERS = 2
PRETRAIN_EPOCHS = 750*2
LAB_FINETUNE_EPOCHS = 200
FIELD_FINETUNE_EPOCHS = 200
Y_SAMPLING = None

In [30]:
trainmode = TRAINMODE
pairing = PAIRING
network = NETWORK
temperature = TEMPERATURE
joint = JOINT
pretrain_skip = PRETRAIN_SKIP
activities = FINETUNE_SKIP
batch_size = BATCH_SIZE
num_workers = NUM_WORKERS
pretrain_epochs = PRETRAIN_EPOCHS 
lab_finetune_epochs = LAB_FINETUNE_EPOCHS 
field_finetune_epochs = FIELD_FINETUNE_EPOCHS
regularize = REGULARIZE 
t = TEMPERATURE
y_sampling = Y_SAMPLING

temperature_dict = {'L':0.1,'M':0.5,'H':1}    
temperature = temperature_dict[t]

if TRAINMODE == 'pwr':
    joint = 'first'

if TRAINMODE == 'normal':
    if regularize == True:
        t = 'r'
    else:
        t = ''
    pairing = 'nuc2'
    freeze=False
else:
    freeze=True

if JOINT == 'joint':
    j='-j'
else:
    j=''
    
if EXTRA == None:
    extra = ''
else:
    extra = '-Extra-'+str(EXTRA)


exp_name = f'Trainmode-{trainmode}-{t}_Network-{network}_Data-exp4csi{pairing}{j}{extra}'
model_outpath  = OUT_PATH+'/'+'saved_models'
record_outpath = OUT_PATH+'/'+'records'

print(exp_name)

Trainmode-simclr-H_Network-vgg16_Data-exp4csinuc2


In [17]:
####################################################### Main #############################################################

datas = import_pair_data(fp4,modal=['csi',pairing])

Importing Data >>>>>>> Complete


In [18]:
X1,X2,y = initial_filtering_activities(datas,activities=pretrain_skip)

In [19]:
X1_train, X1_test, X2_train, X2_test, y_train_, y_test_ = split_datasets(X1,X2,y,split=0.8,stratify=y)

In [31]:
################################ Lab-Pretrain-phase ################################

### data
pretrain_loader = create_dataloader(X1_train,X2_train,batch_size=batch_size,num_workers=num_workers)
print('X1_train: ',X1_train.shape,'\tX2_train: ',X2_train.shape)

### network
encoder, outsize = create_encoder(network,'nuc2')
encoder2, outsize2 = None,None

if joint == 'joint':
    simclr = add_SimCLR(encoder, outsize)
else:
    encoder2, outsize2 = create_encoder(network,pairing)
    simclr = add_SimCLR_multi(enc1=encoder,enc2=encoder2,out_size1=outsize,out_size2=outsize2)
    
    
    
    

### pretraining
phase = 'pretrain'
if trainmode == 'simclr':
    criterion = NT_Xent(batch_size, temperature, world_size=1)
    optimizer = torch.optim.SGD(list(simclr.parameters()), lr=0.0005)
    simclr, record = pretrain(model=simclr,
                              train_loader=pretrain_loader,
                              criterion=criterion,
                              optimizer=optimizer,
                              end=pretrain_epochs,
                              device=device)
    record_log(record_outpath,exp_name,phase,record=record)
    
# save
encoder_fp = save_model(model_outpath,exp_name,phase,simclr.encoder)
del simclr,encoder,outsize, encoder2,outsize2
torch.cuda.empty_cache()

X1_train:  (608, 1, 65, 501) 	X2_train:  (608, 1, 65, 501)
Start Training
Epoch 1: >>>>>>>>> loss: 5.1134161949157715 
Epoch 2: >>>>>>>>> loss: 4.949114799499512 
Epoch 3: >>>>>>>>> loss: 4.887993335723877 
Epoch 4: >>>>>>>>> loss: 4.863486289978027 
Epoch 5: >>>>>>>>> loss: 4.85258150100708 
Epoch 6: >>>>>>>>> loss: 4.847887992858887 
Epoch 7: >>>>>>>>> loss: 4.845532417297363 
Epoch 8: >>>>>>>>> loss: 4.844919204711914 
Epoch 9: >>>>>>>>> loss: 4.844048500061035 
Epoch 10: >>>>>>>>> loss: 4.843818664550781 
Epoch 11: >>>>>>>>> loss: 4.8434600830078125 
Epoch 12: >>>>>>>>> loss: 4.843499183654785 
Epoch 13: >>>>>>>>> loss: 4.843264579772949 
Epoch 14: >>>>>>>>> loss: 4.843740940093994 
Epoch 15: >>>>>>>>> loss: 4.843804359436035 
Epoch 16: >>>>>>>>> loss: 4.843432903289795 
Epoch 17: >>>>>>>>> loss: 4.843632221221924 
Epoch 18: >>>>>>>>> loss: 4.842859268188477 
Epoch 19: >>>>>>>>> loss: 4.843052387237549 
Epoch 20: >>>>>>>>> loss: 4.843655109405518 
Epoch 21: >>>>>>>>> loss: 4.843111

Epoch 180: >>>>>>>>> loss: 4.434841156005859 
Epoch 181: >>>>>>>>> loss: 4.471540451049805 
Epoch 182: >>>>>>>>> loss: 4.467726230621338 
Epoch 183: >>>>>>>>> loss: 4.493007659912109 
Epoch 184: >>>>>>>>> loss: 4.470339775085449 
Epoch 185: >>>>>>>>> loss: 4.501652717590332 
Epoch 186: >>>>>>>>> loss: 4.507394790649414 
Epoch 187: >>>>>>>>> loss: 4.465092658996582 
Epoch 188: >>>>>>>>> loss: 4.4219489097595215 
Epoch 189: >>>>>>>>> loss: 4.541524410247803 
Epoch 190: >>>>>>>>> loss: 4.460423946380615 
Epoch 191: >>>>>>>>> loss: 4.461698532104492 
Epoch 192: >>>>>>>>> loss: 4.4746317863464355 
Epoch 193: >>>>>>>>> loss: 4.558793067932129 
Epoch 194: >>>>>>>>> loss: 4.4239501953125 
Epoch 195: >>>>>>>>> loss: 4.5132880210876465 
Epoch 196: >>>>>>>>> loss: 4.442867755889893 
Epoch 197: >>>>>>>>> loss: 4.407537460327148 
Epoch 198: >>>>>>>>> loss: 4.406606674194336 
Epoch 199: >>>>>>>>> loss: 4.537044048309326 
Epoch 200: >>>>>>>>> loss: 4.393465042114258 
Epoch 201: >>>>>>>>> loss: 4.4315

Epoch 358: >>>>>>>>> loss: 4.492819309234619 
Epoch 359: >>>>>>>>> loss: 4.289132595062256 
Epoch 360: >>>>>>>>> loss: 4.385590553283691 
Epoch 361: >>>>>>>>> loss: 4.2837138175964355 
Epoch 362: >>>>>>>>> loss: 4.276803493499756 
Epoch 363: >>>>>>>>> loss: 4.343650817871094 
Epoch 364: >>>>>>>>> loss: 4.286486625671387 
Epoch 365: >>>>>>>>> loss: 4.405742168426514 
Epoch 366: >>>>>>>>> loss: 4.477736949920654 
Epoch 367: >>>>>>>>> loss: 4.295819282531738 
Epoch 368: >>>>>>>>> loss: 4.331745624542236 
Epoch 369: >>>>>>>>> loss: 4.2684221267700195 
Epoch 370: >>>>>>>>> loss: 4.288593769073486 
Epoch 371: >>>>>>>>> loss: 4.324437141418457 
Epoch 372: >>>>>>>>> loss: 4.283929824829102 
Epoch 373: >>>>>>>>> loss: 4.301592826843262 
Epoch 374: >>>>>>>>> loss: 4.314024448394775 
Epoch 375: >>>>>>>>> loss: 4.2586283683776855 
Epoch 376: >>>>>>>>> loss: 4.25987434387207 
Epoch 377: >>>>>>>>> loss: 4.383795261383057 
Epoch 378: >>>>>>>>> loss: 4.307747840881348 
Epoch 379: >>>>>>>>> loss: 4.306

Epoch 536: >>>>>>>>> loss: 4.21198844909668 
Epoch 537: >>>>>>>>> loss: 4.283807277679443 
Epoch 538: >>>>>>>>> loss: 4.213207721710205 
Epoch 539: >>>>>>>>> loss: 4.205941677093506 
Epoch 540: >>>>>>>>> loss: 4.246153354644775 
Epoch 541: >>>>>>>>> loss: 4.2211833000183105 
Epoch 542: >>>>>>>>> loss: 4.210225582122803 
Epoch 543: >>>>>>>>> loss: 4.228386878967285 
Epoch 544: >>>>>>>>> loss: 4.200644016265869 
Epoch 545: >>>>>>>>> loss: 4.212796688079834 
Epoch 546: >>>>>>>>> loss: 4.278421401977539 
Epoch 547: >>>>>>>>> loss: 4.336925029754639 
Epoch 548: >>>>>>>>> loss: 4.257570266723633 
Epoch 549: >>>>>>>>> loss: 4.299280166625977 
Epoch 550: >>>>>>>>> loss: 4.223015785217285 
Epoch 551: >>>>>>>>> loss: 4.2548747062683105 
Epoch 552: >>>>>>>>> loss: 4.239819049835205 
Epoch 553: >>>>>>>>> loss: 4.274460315704346 
Epoch 554: >>>>>>>>> loss: 4.234482765197754 
Epoch 555: >>>>>>>>> loss: 4.211643695831299 
Epoch 556: >>>>>>>>> loss: 4.2365570068359375 
Epoch 557: >>>>>>>>> loss: 4.288

Epoch 714: >>>>>>>>> loss: 4.176117897033691 
Epoch 715: >>>>>>>>> loss: 4.176149368286133 
Epoch 716: >>>>>>>>> loss: 4.183109283447266 
Epoch 717: >>>>>>>>> loss: 4.230783462524414 
Epoch 718: >>>>>>>>> loss: 4.17239236831665 
Epoch 719: >>>>>>>>> loss: 4.163080215454102 
Epoch 720: >>>>>>>>> loss: 4.20794677734375 
Epoch 721: >>>>>>>>> loss: 4.177829265594482 
Epoch 722: >>>>>>>>> loss: 4.231327533721924 
Epoch 723: >>>>>>>>> loss: 4.162755012512207 
Epoch 724: >>>>>>>>> loss: 4.202054500579834 
Epoch 725: >>>>>>>>> loss: 4.254675388336182 
Epoch 726: >>>>>>>>> loss: 4.1991801261901855 
Epoch 727: >>>>>>>>> loss: 4.192988872528076 
Epoch 728: >>>>>>>>> loss: 4.171797752380371 
Epoch 729: >>>>>>>>> loss: 4.2187299728393555 
Epoch 730: >>>>>>>>> loss: 4.20766544342041 
Epoch 731: >>>>>>>>> loss: 4.225102424621582 
Epoch 732: >>>>>>>>> loss: 4.247033596038818 
Epoch 733: >>>>>>>>> loss: 4.189988613128662 
Epoch 734: >>>>>>>>> loss: 4.1795430183410645 
Epoch 735: >>>>>>>>> loss: 4.19382

Epoch 892: >>>>>>>>> loss: 4.207729339599609 
Epoch 893: >>>>>>>>> loss: 4.186400413513184 
Epoch 894: >>>>>>>>> loss: 4.198369026184082 
Epoch 895: >>>>>>>>> loss: 4.154539585113525 
Epoch 896: >>>>>>>>> loss: 4.159797191619873 
Epoch 897: >>>>>>>>> loss: 4.194396495819092 
Epoch 898: >>>>>>>>> loss: 4.156309127807617 
Epoch 899: >>>>>>>>> loss: 4.137544631958008 
Epoch 900: >>>>>>>>> loss: 4.256815433502197 
Epoch 901: >>>>>>>>> loss: 4.13988733291626 
Epoch 902: >>>>>>>>> loss: 4.130758285522461 
Epoch 903: >>>>>>>>> loss: 4.205866813659668 
Epoch 904: >>>>>>>>> loss: 4.118140697479248 
Epoch 905: >>>>>>>>> loss: 4.20536470413208 
Epoch 906: >>>>>>>>> loss: 4.22907018661499 
Epoch 907: >>>>>>>>> loss: 4.209577560424805 
Epoch 908: >>>>>>>>> loss: 4.177648067474365 
Epoch 909: >>>>>>>>> loss: 4.1849470138549805 
Epoch 910: >>>>>>>>> loss: 4.190554618835449 
Epoch 911: >>>>>>>>> loss: 4.154597759246826 
Epoch 912: >>>>>>>>> loss: 4.186229705810547 
Epoch 913: >>>>>>>>> loss: 4.1541953

Epoch 1069: >>>>>>>>> loss: 4.207704067230225 
Epoch 1070: >>>>>>>>> loss: 4.122988224029541 
Epoch 1071: >>>>>>>>> loss: 4.129924774169922 
Epoch 1072: >>>>>>>>> loss: 4.148612976074219 
Epoch 1073: >>>>>>>>> loss: 4.1358842849731445 
Epoch 1074: >>>>>>>>> loss: 4.145463466644287 
Epoch 1075: >>>>>>>>> loss: 4.158267974853516 
Epoch 1076: >>>>>>>>> loss: 4.147045135498047 
Epoch 1077: >>>>>>>>> loss: 4.169814586639404 
Epoch 1078: >>>>>>>>> loss: 4.131767749786377 
Epoch 1079: >>>>>>>>> loss: 4.105206489562988 
Epoch 1080: >>>>>>>>> loss: 4.156933784484863 
Epoch 1081: >>>>>>>>> loss: 4.142833709716797 
Epoch 1082: >>>>>>>>> loss: 4.157782077789307 
Epoch 1083: >>>>>>>>> loss: 4.129022121429443 
Epoch 1084: >>>>>>>>> loss: 4.167804718017578 
Epoch 1085: >>>>>>>>> loss: 4.1888651847839355 
Epoch 1086: >>>>>>>>> loss: 4.156761646270752 
Epoch 1087: >>>>>>>>> loss: 4.135303020477295 
Epoch 1088: >>>>>>>>> loss: 4.155699253082275 
Epoch 1089: >>>>>>>>> loss: 4.121758460998535 
Epoch 1090:

Epoch 1243: >>>>>>>>> loss: 4.1512770652771 
Epoch 1244: >>>>>>>>> loss: 4.166515350341797 
Epoch 1245: >>>>>>>>> loss: 4.082803726196289 
Epoch 1246: >>>>>>>>> loss: 4.11871337890625 
Epoch 1247: >>>>>>>>> loss: 4.148172855377197 
Epoch 1248: >>>>>>>>> loss: 4.124547481536865 
Epoch 1249: >>>>>>>>> loss: 4.114004611968994 
Epoch 1250: >>>>>>>>> loss: 4.162464141845703 
Epoch 1251: >>>>>>>>> loss: 4.094006538391113 
Epoch 1252: >>>>>>>>> loss: 4.102819919586182 
Epoch 1253: >>>>>>>>> loss: 4.102827072143555 
Epoch 1254: >>>>>>>>> loss: 4.124642372131348 
Epoch 1255: >>>>>>>>> loss: 4.2264580726623535 
Epoch 1256: >>>>>>>>> loss: 4.1874237060546875 
Epoch 1257: >>>>>>>>> loss: 4.162500858306885 
Epoch 1258: >>>>>>>>> loss: 4.165586948394775 
Epoch 1259: >>>>>>>>> loss: 4.138925552368164 
Epoch 1260: >>>>>>>>> loss: 4.115109443664551 
Epoch 1261: >>>>>>>>> loss: 4.100342750549316 
Epoch 1262: >>>>>>>>> loss: 4.126791954040527 
Epoch 1263: >>>>>>>>> loss: 4.125051975250244 
Epoch 1264: >>

Epoch 1417: >>>>>>>>> loss: 4.1097612380981445 
Epoch 1418: >>>>>>>>> loss: 4.134119987487793 
Epoch 1419: >>>>>>>>> loss: 4.121790885925293 
Epoch 1420: >>>>>>>>> loss: 4.106600284576416 
Epoch 1421: >>>>>>>>> loss: 4.08575963973999 
Epoch 1422: >>>>>>>>> loss: 4.101492881774902 
Epoch 1423: >>>>>>>>> loss: 4.111204147338867 
Epoch 1424: >>>>>>>>> loss: 4.183691501617432 
Epoch 1425: >>>>>>>>> loss: 4.128421306610107 
Epoch 1426: >>>>>>>>> loss: 4.170060634613037 
Epoch 1427: >>>>>>>>> loss: 4.1042561531066895 
Epoch 1428: >>>>>>>>> loss: 4.141084671020508 
Epoch 1429: >>>>>>>>> loss: 4.106239318847656 
Epoch 1430: >>>>>>>>> loss: 4.159917831420898 
Epoch 1431: >>>>>>>>> loss: 4.120611190795898 
Epoch 1432: >>>>>>>>> loss: 4.1141581535339355 
Epoch 1433: >>>>>>>>> loss: 4.094602108001709 
Epoch 1434: >>>>>>>>> loss: 4.073763370513916 
Epoch 1435: >>>>>>>>> loss: 4.1050920486450195 
Epoch 1436: >>>>>>>>> loss: 4.101385593414307 
Epoch 1437: >>>>>>>>> loss: 4.119731426239014 
Epoch 1438

In [32]:
################################ Lab-Finetuning-phase ################################
samplings = [1,5,10,'weight','undersampling','oversampling',]
#samplings = ['undersampling','oversampling']

inital = {'lab':True,'field':True}
for sampling in samplings:
    
    print('\n\nsampling: ',sampling,'\n')
    
    ### Sampling data ###
    X_train, X_test, y_train, y_test = select_train_test_dataset(X1_train, X1_test, X2_train, X2_test, y_train_, y_test_, joint)
    X_train, X_test, y_train, y_test, lb = filtering_activities_and_label_encoding(X_train, X_test, y_train, y_test, activities)
    lab_finetune_loader, lab_validatn_loader, class_weight = combine1(X_train, X_test, y_train, y_test, 
                                                                      sampling, lb, batch_size, num_workers, 
                                                                      y_sampling=y_sampling)
    print("class: ",lb.classes_)
    print("class_size: ",1-class_weight)
    
    ### model 
    encoder, outsize = load_encoder(network, encoder_fp)
    model = add_classifier(encoder,in_size=outsize,out_size=len(lb.classes_),freeze=freeze)
    
    # initialization
    phase = 'lab-initial'
    if inital['lab']:
        cmtx,cls = evaluation(model,lab_finetune_loader,label_encoder=lb)
        record_log(record_outpath,exp_name,phase,cmtx=cmtx,cls=cls)
        inital['lab'] = False
    
    # finetuning 
    phase = 'lab-finetune'+'-'+str(sampling)
    criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
    model, record = train(model=model,
                          train_loader= lab_finetune_loader,
                          criterion=criterion,
                          optimizer=optimizer,
                          end= lab_finetune_epochs,
                          test_loader = lab_validatn_loader,
                          device = device,
                          regularize = regularize)
    
    # record and save
    cmtx,cls = evaluation(model,lab_validatn_loader,label_encoder=lb)
    record_log(record_outpath,exp_name,phase,record=record,cmtx=cmtx,cls=cls,acc_rec=True)
    
    if sampling != 'weight':

        del encoder,model,criterion,optimizer# ,record,cmtx,cls
        torch.cuda.empty_cache()

    elif sampling == 'weight':
        
        model_fp = save_model(model_outpath,exp_name,phase,model)
        del encoder,model,criterion,optimizer# ,record,cmtx,cls
        torch.cuda.empty_cache()
    



sampling:  1 

X_train:  (7, 1, 65, 501) 	y_train:  (7,) 	X_test:  (153, 1, 65, 501) 	y_test:  (153,)
class:  ['lay' 'pickup' 'sit' 'stand' 'standff' 'walk' 'waving']
class_size:  tensor([0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429])
             0    1    2    3         4    5    6  accuracy  macro avg  \
f1-score   1.0  0.0  0.0  0.0  0.333333  0.0  0.0  0.285714   0.190476   
precision  1.0  0.0  0.0  0.0  0.200000  0.0  0.0  0.285714   0.171429   
recall     1.0  0.0  0.0  0.0  1.000000  0.0  0.0  0.285714   0.285714   
support    1.0  1.0  1.0  1.0  1.000000  1.0  1.0  0.285714   7.000000   

           weighted avg  
f1-score       0.190476  
precision      0.171429  
recall         0.285714  
support        7.000000  
Start Training
Epoch 1: 

/home/andy/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


> loss: 1.9560297727584839 
Epoch 2: > loss: 1.890652060508728 
Epoch 3: > loss: 1.8159633874893188 
Epoch 4: > loss: 1.7556538581848145 
Epoch 5: > loss: 1.7002261877059937 
Epoch 6: > loss: 1.6365885734558105 
Epoch 7: > loss: 1.5810446739196777 
Epoch 8: > loss: 1.4592682123184204 
Epoch 9: > loss: 1.3953380584716797 
Epoch 10: > loss: 1.3290945291519165  accuracy: 0.3464052287581699
Epoch 11: 

/home/andy/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


> loss: 1.2323356866836548 
Epoch 12: > loss: 1.221384882926941 
Epoch 13: > loss: 1.1361382007598877 
Epoch 14: > loss: 1.0930109024047852 
Epoch 15: > loss: 0.9979800581932068 
Epoch 16: > loss: 0.9375760555267334 
Epoch 17: > loss: 0.8609657287597656 
Epoch 18: > loss: 0.7998826503753662 
Epoch 19: > loss: 0.814443051815033 
Epoch 20: > loss: 0.7892422080039978  accuracy: 0.41830065359477125
Epoch 21: > loss: 0.6564591526985168 
Epoch 22: > loss: 0.6102048754692078 
Epoch 23: > loss: 0.6314747333526611 
Epoch 24: > loss: 0.5235037207603455 
Epoch 25: > loss: 0.5253557562828064 
Epoch 26: > loss: 0.5180931687355042 
Epoch 27: > loss: 0.5057271122932434 
Epoch 28: > loss: 0.41560542583465576 
Epoch 29: > loss: 0.4145640432834625 
Epoch 30: > loss: 0.4506375789642334  accuracy: 0.40522875816993464
Epoch 31: > loss: 0.4271174669265747 
Epoch 32: > loss: 0.3124985694885254 
Epoch 33: > loss: 0.3765183985233307 
Epoch 34: > loss: 0.28491878509521484 
Epoch 35: > loss: 0.318364679813385 
E

...

/home/andy/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


>>>>>>>>> loss: 1.0566500425338745 
Epoch 12: >>>>>>>>> loss: 1.005725622177124 
Epoch 13: >>>>>>>>> loss: 0.934981107711792 
Epoch 14: >>>>>>>>> loss: 0.8753644227981567 
Epoch 15: >>>>>>>>> loss: 1.0586923360824585 
Epoch 16: >>>>>>>>> loss: 0.7974610328674316 
Epoch 17: >>>>>>>>> loss: 0.9216253757476807 
Epoch 18: >>>>>>>>> loss: 0.765857458114624 
Epoch 19: >>>>>>>>> loss: 0.9344719648361206 
Epoch 20: >>>>>>>>> loss: 0.7765089273452759  accuracy: 0.6209150326797386
Epoch 21: >>>>>>>>> loss: 0.7393068671226501 
Epoch 22: >>>>>>>>> loss: 0.7529311776161194 
Epoch 23: >>>>>>>>> loss: 0.8624997735023499 
Epoch 24: >>>>>>>>> loss: 0.9145192503929138 
Epoch 25: >>>>>>>>> loss: 0.8180721998214722 
Epoch 26: >>>>>>>>> loss: 0.6564581990242004 
Epoch 27: >>>>>>>>> loss: 0.7933535575866699 
Epoch 28: >>>>>>>>> loss: 0.7751258611679077 
Epoch 29: >>>>>>>>> loss: 0.6151537895202637 
Epoch 30: >>>>>>>>> loss: 0.7147831320762634  accuracy: 0.6797385620915033
Epoch 31: >>>>>>>>> loss: 0.6770121

Epoch 177: >>>>>>>>> loss: 0.23051893711090088 
Epoch 178: >>>>>>>>> loss: 0.16391751170158386 
Epoch 179: >>>>>>>>> loss: 0.19658517837524414 
Epoch 180: >>>>>>>>> loss: 0.19467151165008545  accuracy: 0.7647058823529411
Epoch 181: >>>>>>>>> loss: 0.14210794866085052 
Epoch 182: >>>>>>>>> loss: 0.1731618344783783 
Epoch 183: >>>>>>>>> loss: 0.1463361233472824 
Epoch 184: >>>>>>>>> loss: 0.1286555677652359 
Epoch 185: >>>>>>>>> loss: 0.18111920356750488 
Epoch 186: >>>>>>>>> loss: 0.13178449869155884 
Epoch 187: >>>>>>>>> loss: 0.13284046947956085 
Epoch 188: >>>>>>>>> loss: 0.1088583767414093 
Epoch 189: >>>>>>>>> loss: 0.14102962613105774 
Epoch 190: >>>>>>>>> loss: 0.16100023686885834  accuracy: 0.7581699346405228
Epoch 191: >>>>>>>>> loss: 0.13904421031475067 
Epoch 192: >>>>>>>>> loss: 0.1695171594619751 
Epoch 193: >>>>>>>>> loss: 0.1128179281949997 
Epoch 194: >>>>>>>>> loss: 0.10864844918251038 
Epoch 195: >>>>>>>>> loss: 0.187486931681633 
Epoch 196: >>>>>>>>> loss: 0.162641689

Epoch 141: >>> loss: 0.4230392575263977 
Epoch 142: >>> loss: 0.47159796953201294 
Epoch 143: >>> loss: 0.42790743708610535 
Epoch 144: >>> loss: 0.4089430868625641 
Epoch 145: >>> loss: 0.39503979682922363 
Epoch 146: >>> loss: 0.42614859342575073 
Epoch 147: >>> loss: 0.39974889159202576 
Epoch 148: >>> loss: 0.31440359354019165 
Epoch 149: >>> loss: 0.42989519238471985 
Epoch 150: >>> loss: 0.3752046525478363  accuracy: 0.6209150326797386
Epoch 151: >>> loss: 0.41830843687057495 
Epoch 152: >>> loss: 0.3796570599079132 
Epoch 153: >>> loss: 0.44533810019493103 
Epoch 154: >>> loss: 0.382802814245224 
Epoch 155: >>> loss: 0.35854029655456543 
Epoch 156: >>> loss: 0.3914303779602051 
Epoch 157: >>> loss: 0.3980827033519745 
Epoch 158: >>> loss: 0.3636191189289093 
Epoch 159: >>> loss: 0.37008774280548096 
Epoch 160: >>> loss: 0.39569681882858276  accuracy: 0.6339869281045751
Epoch 161: >>> loss: 0.3897232115268707 
Epoch 162: >>> loss: 0.3159986436367035 
Epoch 163: >>> loss: 0.348528

Epoch 76: >>>>>>>>>>>>>>>>>>>>> loss: 0.12711261212825775 
Epoch 77: >>>>>>>>>>>>>>>>>>>>> loss: 0.12137028574943542 
Epoch 78: >>>>>>>>>>>>>>>>>>>>> loss: 0.09368198364973068 
Epoch 79: >>>>>>>>>>>>>>>>>>>>> loss: 0.15299682319164276 
Epoch 80: >>>>>>>>>>>>>>>>>>>>> loss: 0.1863972246646881  accuracy: 0.7189542483660131
Epoch 81: >>>>>>>>>>>>>>>>>>>>> loss: 0.12294496595859528 
Epoch 82: >>>>>>>>>>>>>>>>>>>>> loss: 0.0877305343747139 
Epoch 83: >>>>>>>>>>>>>>>>>>>>> loss: 0.11367420107126236 
Epoch 84: >>>>>>>>>>>>>>>>>>>>> loss: 0.08494868874549866 
Epoch 85: >>>>>>>>>>>>>>>>>>>>> loss: 0.07822626084089279 
Epoch 86: >>>>>>>>>>>>>>>>>>>>> loss: 0.0817377120256424 
Epoch 87: >>>>>>>>>>>>>>>>>>>>> loss: 0.1199239045381546 
Epoch 88: >>>>>>>>>>>>>>>>>>>>> loss: 0.12116186320781708 
Epoch 89: >>>>>>>>>>>>>>>>>>>>> loss: 0.1104278489947319 
Epoch 90: >>>>>>>>>>>>>>>>>>>>> loss: 0.1079699769616127  accuracy: 0.7320261437908496
Epoch 91: >>>>>>>>>>>>>>>>>>>>> loss: 0.06709301471710205 
Epoc

In [ ]:
assert False

# Records

In [ ]:
# simclr

       'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'  

'full'  11         33         12         19                    99
'full'  12         34         11         19                    98         57
'full'  9          32         13         19          5         99
'full'  7          27         13         17          6         95         57    
    
#vgg16             #joint
'1pcs'  0.48       0.485981   0.153846   0.478873               0.805825
'5pcs'  0.062500   0.536082   0.410256   0.307692               0.931217
'10pcs' 0.612245   0.349206   0.354430   0.562500               0.953368
'full'  0.687500   0.666667   0.428571   0.622951               0.99

'1pcs'  0.512821   0.366197   0.400000   0.482759               0.563536   0.493151
'5pcs'  0.5        0.358974   0.400000   0.422535               0.666667   0.496000
'10pcs' 0.727273   0.623377   0.405797   0.448276               0.649485   0.521739 
'full'  0.727273   0.673267   0.407407   0.633333               0.942308   0.919355

'1pcs'  0.219178   0.041667   0.186047   0.506329    0.12500    0.613757
'5pcs'  0.461538   0.578947   0.318841   0.375000    0.342857   0.954315 
'10pcs' 0.387097   0.488372   0.493506   0.520000    0.461538   0.964467 
'full'  0.514286   0.666667   0.448276   0.622951    0.370370   0.975369

'1pcs'  0.214286   0.378947   0.2500     0.347826    0.272727   0.283688   0.457447
'5pcs'  0.228571   0.595745   0.2500     0.509091    0.448980   0.520833   0.359712
'10pcs' 0.5        0.539326   0.412698   0.400000    0.424242   0.682927   0.512397
'full'  0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721

In [ ]:
# simclr

        'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving' 
    
#vgg16             #first'    #L
'1pcs'  0.0        0.638297   0.3783783  0.222222    0.142857   0.328767   0.479166 
'5pcs'  0.526315   0.627450   0.5        0.476190    0.133333   0.66       0.441176
'10pcs' 0.777777   0.651162   0.5714285  0.5         0.526315   0.64       0.578947
'full'  0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821
'under' 0.700000   0.714286   0.533333   0.482759    0.421053   0.755102   0.617647
'over'  0.736842   0.681818   0.647059   0.642857    0.461538   0.88       0.823529
                                
                              #M
'full'  0.571429   0.653061   0.666667   0.538462    0.631579   0.921569  0.878788
'under' 0.555556   0.600000   0.578947   0.500000    0.583333   0.835165  0.760563
'over'  0.625000   0.723404   0.600000   0.533333    0.823529   0.875000  0.828571

                              #H
'full'  0.428571   0.680851   0.647059   0.592593    0.588235   0.847826  0.826667 
'under' 0.526316   0.590909   0.571429   0.592593    0.608696   0.727273 0.675325
'over'  0.705882   0.638298   0.600000   0.571429    0.631579   0.891304  0.876712

                   #joint
'1pcs'  0.214286   0.378947   0.2500     0.347826    0.272727   0.283688   0.457447
'5pcs'  0.228571   0.595745   0.2500     0.509091    0.448980   0.520833   0.359712
'10pcs' 0.5        0.539326   0.412698   0.400000    0.424242   0.682927   0.512397
'full'  0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721

                   #pwr
'full'  0.0        0.775510   0.588235   0.666667    0.0        0.944000   0.833333
'under' 0.4        0.618182   0.705882   0.583333    0.25       0.520833   0.368421
'over'  0.285714   0.760000   0.500000   0.666667    0.40       0.918033   0.806452


        

#shallow           #joint
'1pcs'  0.384615   0.037736   0.356164   0.046512    0.0        0.555556   0.268657
'5pcs'  0.146341   0.169492   0.156863   0.484211    0.266667   0.357616   0.470588
'10pcs' 0.408163   0.391753   0.222222   0.338983    0.186047   0.541935   0.468750
'full'  0.333333   0.444444   0.356164   0.406780    0.370370   0.94       0.892086

                   #first
'full'  0.500000   0.5        0.4375     0.580645    0.615385   0.923077   0.909091  
'under' 0.384615   0.514286   0.685714   0.787879    0.352941   0.800000   0.769231
'over'  0.400000   0.465116   0.470588   0.580645    0.571429   0.812500   0.794521


#alexnet
'1pcs'  0.0        0.068182   0.170213   0.421053    0.0        0.375940   0.551351
'5pcs'  0.274510   0.515464   0.393939   0.285714    0.166667   0.590909   0.525547
'10pcs' 0.615385   0.530120   0.41791    0.285714    0.476190   0.604651   0.575163 
'full'  0.592593   0.606061   0.433333   0.483871    0.5        0.920792   0.876923
 
                   #first
'full'  0.875000   0.681818   0.611111   0.571429    0.75       0.907216  0.898551
'under' 0.666667   0.615385   0.666667   0.689655    0.695652   0.775510  0.603175
'over'  0.823529   0.604651   0.457143   0.571429    0.777778   0.888889  0.848485

# Resnet           #first

'full'  0.400000   0.666667   0.611111   0.482759    0.428571   0.843137  0.823529
'under' 0.400000   0.450000   0.611111   0.444444    0.315789   0.747253  0.576271
'over'  0.526316   0.487805   0.550000   0.466667    0.307692   0.860215  0.828571

In [ ]:
# normal
        'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'  
#vgg16              #joint
'1pcs'  0.074074   0.192771   0.385542   0.256410    0.0        0.133333   0.055556
'5pcs'  0.378378   0.325581   0.204082   0.425000    0.210526   0.548571   0.359712
'10pcs' 0.0        0.385965   0.372093   0.235294    0.204082   0.205128   0.500000
'full'  0.421053   0.494382   0.327273   0.538462    0.648649   0.919431   0.876923

                    #first
'full'  0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  
'under' 0.500000   0.550000   0.333333   0.400000    0.444444   0.634146   0.588235
'over'  0.571429   0.428571   0.514286   0.666667    0.200      0.747475   0.684211

#shallow           # joint
'1pcs'  0.0        0.275862   0.093023   0.333333    0.133333   0.264706   0.335025
'5pcs'  0.060606   0.368421   0.354839   0.307692    0.266667   0.560847   0.382166
'10pcs' 0.500000   0.346667   0.333333   0.489362    0.466667   0.663317   0.484375
'full'  0.518519   0.574468   0.459016   0.606061    0.444444   0.941176   0.932331

                    #first
'full'  0.571429   0.448980   0.424242   0.500000    0.500      0.938776   0.916667
'under' 0.631579   0.342857   0.545455   0.578947    0.500000   0.786517   0.666667
'over'  0.461538   0.372093   0.526316   0.580645    0.400000   0.969072   0.927536

#alexnet
'1pcs'  0.0        0.038462   0.311475   0.0         0.160000   0.472527   0.301370  
'5pcs'  0.238806   0.338028   0.323529   0.195122    0.162162   0.380952   0.444444
'10pcs' 0.430769   0.296296   0.301887   0.507937    0.338983   0.589744   0.488889
'full'  0.645161   0.529412   0.327273   0.428571    0.514286   0.915423   0.878788
 
        #first'
'full'  0.500000   0.553191   0.388889   0.500000    0.428571   0.893617   0.788732
'under' 0.470588   0.476190   0.411765   0.482759    0.526316   0.704545   0.649351
'over'  0.714286   0.553191   0.4375     0.500000    0.714286   0.909091   0.777778

# Resnet           #first

'full'  0.400000   0.692308   0.500000   0.516129    0.571429   0.979592   0.941176
'under' 0.222222   0.564103   0.470588   0.461538    0.461538   0.758621   0.698413
'over'  0.166667   0.304348   0.375      0.486486    0.421053   0.698795   0.707692

In [ ]:
# overall 


'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'simclr'  'first'  #vgg16
0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  'normal'    
0.421053   0.494382   0.327273   0.538462    0.648649   0.919431   0.876923            'joint'
0.500000   0.619048   0.600000   0.533333    0.400000   0.769231   0.724638            'l2reg'  


# comparison between models: larger model not always the best, clr 

'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.571428   0.448979   0.424242   0.5         0.5        0.938775   0.916666  'shallow'  'normal'
0.500000   0.5        0.4375     0.580645    0.615385   0.923077   0.909091  'shallow'  'simclr' 

0.5        0.553191   0.388888   0.5         0.428571   0.893617   0.788732  'alexnet'  'normal'
0.875000   0.681818   0.611111   0.571429    0.75       0.907216   0.898551  'alexnet'  'simclr'

0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  'vgg16'    'normal'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'vgg16'    'simclr'

0.400000   0.692308   0.500000   0.516129    0.571429   0.979592   0.941176  'resnet'   'normal'
0.400000   0.666667   0.611111   0.482759    0.428571   0.843137   0.823529  'resnet'   'simclr'


# effect on sample: simclr significantly improve sample efficiency 

'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.210526   0.133333   0.2285714  0.266666    0.0        0.594059   0.148148  '1pcs'  'normal'   #vgg16
0.0,       0.638297   0.3783783  0.222222    0.142857   0.328767   0.479166  '1pcs'  'simclr'   

0.125      0.384615   0.3999999  0.32        0.230769   0.536082   0.369230  '5pcs'  'normal'
0.526315   0.627450   0.5        0.476190    0.133333   0.66       0.441176  '5pcs'  'simclr'

0.285714   0.333333   0.4705882  0.538461    0.434782   0.447368   0.430379  '10pcs' 'normal'
0.777777   0.651162   0.5714285  0.5         0.526315   0.64       0.578947  '10pcs' 'simclr'

0.714286   0.415094   0.258065   0.592593    0.363636   0.884615   0.818182  'full'  'normal'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'full'  'simclr'

0.500000   0.550000   0.333333   0.400000    0.444444   0.634146   0.588235  'under' 'normal'
0.700000   0.714286   0.533333   0.482759    0.421053   0.755102   0.617647  'under' 'simclr'

0.571429   0.428571   0.514286   0.666667    0.200      0.747475   0.684211  'over'  'normal'
0.736842   0.681818   0.647059   0.642857    0.461538   0.88       0.823529  'over'  'simclr'


# effect on modality/view: seperate multiview is more beneficial to the training 
'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821  'first'
0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721  'joint'
0.0        0.775510   0.588235   0.666667    0.0        0.944000   0.833333  'pwr'


# effect on temperature: seperate multiview is more beneficial to the training 
'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'
0.666667   0.708333   0.6875     0.592593    0.363636   0.893204   0.835821   'L'
0.571429   0.653061   0.666667   0.538462    0.631579   0.921569   0.878788   'M'
0.428571   0.680851   0.647059   0.592593    0.588235   0.847826   0.826667   'H'


# activity 
'lay'      'pickup'   'sit'      'stand'     'standff'  'walk'    'waving'  

0.687500   0.666667   0.428571   0.622951               0.99                 #'vgg16' 'simclr'
0.727273   0.673267   0.407407   0.633333               0.942308   0.919355
0.514286   0.666667   0.448276   0.622951    0.370370   0.975369
0.437500   0.586957   0.426230   0.557377    0.400000   0.917874   0.883721

# Extra

In [ ]:
################################ Field-Finetuning-phase ################################
    
        for field_sampling in [1,5,10]:

            Xf,yf = import_data(fp2)
            field_finetune_loader, field_validatn_loader, lb = process_field_data(Xf,yf,num=field_sampling,lb=lb)
            
            
            model = load_model(network, freeze, lb, model_fp)
            
            # initialization
            phase = 'field-initial'
            if inital['field'] == True:
                cmtx,cls = evaluation(model,test_loader,label_encoder=lb)
                record_log(record_outpath,exp_name,phase,cmtx=cmtx,cls=cls)
                inital['field'] = False
                
            # finetuning 
            phase = 'field-finetune'
            criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
            optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
            model, record = train(model=model,
                                  train_loader= field_finetune_loader,
                                  criterion=criterion,
                                  optimizer=optimizer,
                                  end= field_finetune_epochs,
                                  test_loader = field_validatn_loader,
                                  device = device,
                                  regularize = regularize)
            # record and save
            evaluation(model,field_validatn_loader,label_encoder=lb)
            record_log(record_outpath,exp_name,phase,record=record,cmtx=cmtx,cls=cls,acc_rec=True)
            del encoder,model,record,cmtx,cls,criterion,optimizer
            torch.cuda.empty_cache()

        
        
        
    
    